'''In Excerise 3.02 Business Hypothesis Testing for Age versus Propensity for a Term Loan

Business Hypothesis Testing to Find Employment Status versus Propensity for Term Deposits
---




1. Formulate the hypothesis between employment status and the propensity for term deposits. Let the hypothesis be as follows: High paying employees prefer term deposits than other categories of employees.


In [4]:
#2. Install and import the necessary libraries such as pandas and altair.
import pandas as pd
import altair as alt

In [5]:
#Assign link to dataset
file_url = 'https://raw.githubusercontent.com/PacktWorkshops/The-Data-Science-Workshop/master/Chapter03/bank-full.csv'
# Loading the data using pandas
bank_data = pd.read_csv(file_url, sep=";")

In [6]:
#Load the first few rows to check the data
bank_data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [7]:
#.shape function is used to find the overall shape of the dataset.
print(bank_data.shape)

(45211, 17)


In [8]:
bank_data.tail()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [9]:
#find the summary of the numerical raw data as a table output using the .describe() 
bank_data.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [10]:
#count the number of records for each age group.
#Take DataFrame, bankData and then filter it for all cases where the term deposit is yes using the mask bankData['y'] == 'yes'. 
filter_mask = bank_data['y'] == 'yes'
#These cases are grouped through the groupby() method and then aggregated according to job through the agg() method. 
#Finally we need to use .reset_index() to get a well-structure DataFrame
bankSub1 = bank_data[filter_mask].groupby('job')['y'].agg(agegrp='count').reset_index()

From the banking DataFrame, bankData, find the distribution of employment status using the .groupby(), .agg() and .reset_index() methods.

Group the data with respect to employment status using the .groupby() method and find the total count of propensities for each employment status using the .agg() method.

In [11]:
#Group the data per age using the groupby() method and find the total number of customers under each job group using the agg() method
# Getting another perspective
jobGrpTotal = bank_data.groupby('job')['y'].agg(jobGrpTotal='count').reset_index()
jobGrpTotal.head()

,job,jobGrpTotal
0,admin.,5171
1,blue-collar,9732
2,entrepreneur,1487
3,housemaid,1240
4,management,9458


In [12]:
#group the data by both job and propensity of purchase and find the total counts under each category of propensity, which are yes and no
# Getting all the details in one place
jobProp = bank_data.groupby(['job','y'])['y'].agg(jobCat='count').reset_index()
jobProp.head()

,job,y,jobCat
0,admin.,no,4540
1,admin.,yes,631
2,blue-collar,no,9024
3,blue-collar,yes,708
4,entrepreneur,no,1364


In [13]:
#Merge both of these DataFrames based on the job variable using the pd.merge() function, 
#and then divide each category of propensity within each job group by the total customers 
#in the respective job group to get the proportion of customers. 
# Merging both the data frames
jobComb = pd.merge(jobProp, jobGrpTotal,left_on = ['job'], right_on=['job'])
jobComb['catProp'] = (jobComb.jobCat/jobComb.jobGrpTotal)*100
jobComb.head()

,job,y,jobCat,jobGrpTotal,catProp
0,admin.,no,4540,5171,87.797331
1,admin.,yes,631,5171,12.202669
2,blue-collar,no,9024,9732,92.725031
3,blue-collar,yes,708,9732,7.274969
4,entrepreneur,no,1364,1487,91.728312


In [14]:
# Visualising the relationship using altair
#Using columns from Datframe noted above for job, newly created column catProp, and y (yes/no)
##alt.Chart(jobComb).mark_bar().encode(alt.X('job',title=''), alt.Y('catProp', title='Propensity Proportion'), color='y')
alt.Chart(jobComb, title="Propensity of purchase by job").mark_bar().encode(alt.X('job',title=''), 
                                     alt.Y('catProp', title='Propensity Proportion'), color=alt.Color('y', legend=alt.Legend(title='')))

alt.Chart(...)

The chart from the solution and noted above are different because altair was used above as done in Exercise 3.02 and solution used dfply, ggplot, and matplotlib.